In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow

mlflow.set_tracking_uri("http://ec2-54-74-30-236.eu-west-1.compute.amazonaws.com:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://mlflow-artifacts-remote-week04/1', experiment_id='1', lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
import os

# os.environ["AWS_ACCESS_KEY_ID"] = "minio"
# os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
# os.environ["MLFLOW_S3_ENDPOINT_URL"] = f"http://minio-server:9000"

In [5]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [6]:
data_path = "../../week1-intro-environment/notebooks/data/"
df_train = read_dataframe(data_path+'green_tripdata_2021-01.parquet')
df_val = read_dataframe(data_path+'green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [7]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = mean_squared_error(y_pred, y_val, squared=False)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, artifact_path="model")

{'max_depth': 20, 'n_estimators': 10, 'min_samples_leaf': 10, 'random_state': 0} 6.766727656130231


In [8]:
from mlflow.tracking import MlflowClient


In [9]:
MLFLOW_TRACKING_URI = 'http://ec2-54-74-30-236.eu-west-1.compute.amazonaws.com:5000'
RUN_ID = 'dc88ab37012042578678bb6ec6e814bb'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [12]:
path = client.download_artifacts(run_id=RUN_ID) #, path='dict_vectorizer.bin')

TypeError: download_artifacts() missing 1 required positional argument: 'path'

In [22]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [23]:
dv

DictVectorizer()

In [13]:
? client.download_artifacts

Signature:
 client.download_artifacts(
    run_id: str,
    path: str,
    dst_path: Optional[str] = None,
) -> str
Docstring:
Download an artifact file or directory from a run to a local directory if applicable,
and return a local path for it.

:param run_id: The run to download artifacts from.
:param path: Relative source path to the desired artifact.
:param dst_path: Absolute path of the local filesystem destination directory to which to
                 download the specified artifacts. This directory must already exist.
                 If unspecified, the artifacts will either be downloaded to a new
                 uniquely-named directory on the local filesystem or will be returned
                 directly in the case of the LocalArtifactRepository.
:return: Local path of desired artifact.

.. code-block:: python
    :caption: Example

    import os
    import mlflow
    from mlflow.tracking import MlflowClient

    features = "rooms, zipcode, median_price, school_rating, tran